In [1]:
import pandas as pd## need to install pandas in enviroment through anaconda
import datetime
from dateutil import parser
from pandas import ExcelWriter###have to install xlsxwriter in anaconda
from selenium import webdriver### need to install selenium in environment
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager###pip install webdriver-manager
import time


### Set chrome and chromedriver locations, if needed to avoid errors (I needed to)


In [2]:
cOptions = webdriver.ChromeOptions()
# Path to chrome.exe
cOptions.binary_location = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_driver_binary = Service("D:\\OMSCS\\CHEM8843-4803\\chromedriver.exe")
### create instance of the webdriver to automatically open webpages
driver = webdriver.Chrome(service=chrome_driver_binary, options=cOptions)
# driver=webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome()
### this can now be used as: driver.get(url) to open webpages.
### Don't forget to close each window with driver.quit() after you're done. Otherwise, you'll open lots of windows as you loop through.

## Kayak typical search
https://www.kayak.com/flights/ATL-JFK/2022-10-01/2022-10-31/?sort=bestflight_a

In [3]:
from_airport = 'LAX'
to_airport = 'JFK'
st_date = datetime.date(2022, 10, 1)
en_date = datetime.date(2022, 10, 31)

url = "https://www.kayak.com/flights/"+ from_airport +"-" + to_airport +"/" + \
        st_date.strftime("%Y-%m-%d") +"/" + en_date.strftime("%Y-%m-%d") + "/?sort=bestflight_a"
driver.get(url)
time.sleep(20)


Inspect element in Chrome
for that thing that is highlighted, copy full xpath. (make sure you are sufficiently far indented in the xpath to only get the price for "Best" fare 
Then search by xpath using the chromedriver instance you created (named "driver" above).

In [4]:
### generate dates for travel, +/- deltaT days on outbound and return
deltaT = datetime.timedelta(days = 3)

xpath_text = "/html/body/div[1]/div[1]/main/div/div[2]/div[2]/div/div[2]/div[1]/div[2]/div[2]/div[4]/div/div/div[2]/div[1]/a[2]/div[1]/div/div/div[2]/span[1]"
# fare = driver.find_element_by_xpath(xpath_text).text
fare = driver.find_element("xpath", xpath_text).text
fare

'$282'

In [5]:
fare

'$364'

### Create a pandas dataframe to hold the results of each search, then write to excel file

In [5]:
%%time
### generate pandas dataframe to hold data
col_names =  ['Origin', 'Destination', 'Departure','Return','Price','URL']
flight_df = pd.DataFrame(columns=col_names)
origin = ['ATL', 'LAX']
destination = ['JFK', 'BWI']
date_range = [-1, 0, 1]

for o in origin:
    for d in destination:
        for i in date_range:
            dep_date = st_date + datetime.timedelta(days=i)
            for j in date_range:
                ret_date = en_date + datetime.timedelta(days=j)
                url = "https://www.kayak.com/flights/"+ o +"-" + d +"/" + \
                       dep_date.strftime("%Y-%m-%d") +"/" + ret_date.strftime("%Y-%m-%d") + "/?sort=bestflight_a"
                print(url)
                driver.get(url)
                time.sleep(20)
                xpath_text = "/html/body/div[1]/div[1]/main/div/div[2]/div[2]/div/div[2]/div[1]/div[2]/div[2]/div[4]/div/div/div[2]/div[1]/a[2]/div[1]/div/div/div[2]/span[1]"

                fare = driver.find_element("xpath", xpath_text).text
                flight_df = flight_df.append(pd.DataFrame([[o, d, dep_date, ret_date, fare, url]], columns=col_names), ignore_index=True)

https://www.kayak.com/flights/ATL-JFK/2022-09-30/2022-10-30/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-09-30/2022-10-31/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-09-30/2022-11-01/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-10-01/2022-10-30/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-10-01/2022-10-31/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-10-01/2022-11-01/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-10-02/2022-10-30/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-10-02/2022-10-31/?sort=bestflight_a
https://www.kayak.com/flights/ATL-JFK/2022-10-02/2022-11-01/?sort=bestflight_a
https://www.kayak.com/flights/ATL-BWI/2022-09-30/2022-10-30/?sort=bestflight_a
https://www.kayak.com/flights/ATL-BWI/2022-09-30/2022-10-31/?sort=bestflight_a
https://www.kayak.com/flights/ATL-BWI/2022-09-30/2022-11-01/?sort=bestflight_a
https://www.kayak.com/flights/ATL-BWI/2022-10-01/202

In [6]:
flight_df

,Origin,Destination,Departure,Return,Price,URL
0,ATL,JFK,2022-09-30,2022-10-30,$298,https://www.kayak.com/flights/ATL-JFK/2022-09-...
1,ATL,JFK,2022-09-30,2022-10-31,,https://www.kayak.com/flights/ATL-JFK/2022-09-...
2,ATL,JFK,2022-09-30,2022-11-01,,https://www.kayak.com/flights/ATL-JFK/2022-09-...
3,ATL,JFK,2022-10-01,2022-10-30,,https://www.kayak.com/flights/ATL-JFK/2022-10-...
4,ATL,JFK,2022-10-01,2022-10-31,,https://www.kayak.com/flights/ATL-JFK/2022-10-...
5,ATL,JFK,2022-10-01,2022-11-01,,https://www.kayak.com/flights/ATL-JFK/2022-10-...
6,ATL,JFK,2022-10-02,2022-10-30,,https://www.kayak.com/flights/ATL-JFK/2022-10-...
7,ATL,JFK,2022-10-02,2022-10-31,,https://www.kayak.com/flights/ATL-JFK/2022-10-...
8,ATL,JFK,2022-10-02,2022-11-01,,https://www.kayak.com/flights/ATL-JFK/2022-10-...
9,ATL,BWI,2022-09-30,2022-10-30,$163,https://www.kayak.com/flights/ATL-BWI/2022-09-...


### make sure to quit the chromedriver each time an instance of the chromedriver class is created

In [93]:
driver.quit()